In [56]:
!pip install pydub --quiet
!pip install librosa --quiet
!pip install transformers --quiet

In [57]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
%cd /content/drive/MyDrive/'CSCI 535 Project'/Implementation/training_models/

/content/drive/MyDrive/CSCI 535 Project/Implementation/training_models


In [4]:
import librosa 
from pydub import AudioSegment
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model,
    Wav2Vec2PreTrainedModel,
)

In [44]:
dataset = pd.read_csv('./data/eval_set.csv')
dataset

,video_id,transcript,file_path,label
0,audio_81.wav,I m not hating on men or blaming men I am lite...,/data/nonmisogyny_eval/audio_81.wav,0
1,audio_82.wav,it seems like you guys fight it and I feel lik...,/data/nonmisogyny_eval/audio_82.wav,0
2,audio_83.wav,okay so first of all I don't think its an atta...,/data/nonmisogyny_eval/audio_83.wav,0
3,audio_84.wav,you also write about and this is unplesant stu...,/data/nonmisogyny_eval/audio_84.wav,0
4,audio_85.wav,embattled CNN anchor Don Lemon absent for a th...,/data/nonmisogyny_eval/audio_85.wav,0
5,audio_82.wav,she is wearing her fuck me boots very nice is ...,/data/misogyny_eval/audio_82.wav,1
6,audio_83.wav,He hated girls who sat like boys who acted lik...,/data/misogyny_eval/audio_83.wav,1
7,eval_12.wav,are you saying women cannot do condom hauls? W...,/data/misogyny_eval/eval_12.wav,1
8,eval_5.wav,I don't think the world has ever been equal. I...,/data/misogyny_eval/eval_5.wav,1
9,7NUU24OOh3Y,this video shows what the most popular books o...,/data/nonmisogyny_eval/7NUU24OOh3Y.wav,0


In [16]:
to_generate = list(dataset['video_id'][35:])
labels = list(dataset['label'][35:])

In [17]:
file_path = [] 
for i in range(len(labels)): 
  if labels[i] == 0: 
    file_path.append('/data/nonmisogyny_eval/'+to_generate[i]+'.wav')
  else: 
    file_path.append('/data/misogyny_eval/'+to_generate[i]+'.wav')


In [38]:
file_path[6] = '/data/misogyny_eval/audio_83.wav' 
file_path[5] = '/data/misogyny_eval/audio_82.wav' 

In [40]:
dataset.to_csv('./data/eval_set.csv', index=False)

In [45]:
class RegressionHead(nn.Module):
    r"""Classification head."""

    def __init__(self, config):

        super().__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):

        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)

        return x


class EmotionModel(Wav2Vec2PreTrainedModel):
    r"""Speech emotion classifier."""

    def __init__(self, config):

        super().__init__(config)

        self.config = config
        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = RegressionHead(config)
        self.init_weights()

    def forward(
            self,
            input_values,
    ):

        outputs = self.wav2vec2(input_values)
        hidden_states = outputs[0]
        hidden_states = torch.mean(hidden_states, dim=1)
        logits = self.classifier(hidden_states)

        return hidden_states, logits


In [46]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [47]:
# load model from hub
#device = 'cuda'
model_name = 'audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim'
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = EmotionModel.from_pretrained(model_name).to(device)

# dummy signal
sampling_rate = 16000
signal = np.zeros((1, sampling_rate), dtype=np.float32)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
signal.shape 

(1, 16000)

In [28]:
def process_func(
    x: np.ndarray,
    sampling_rate: int,
    embeddings: bool = False,
) -> np.ndarray:
    r"""Predict emotions or extract embeddings from raw audio signal."""

    # run through processor to normalize signal
    # always returns a batch, so we just get the first entry
    # then we put it on the device
    y = processor(x, sampling_rate=sampling_rate)
    y = y['input_values'][0]
    y = torch.from_numpy(y).to(device)
    print(y.shape)

    # run through model
    with torch.no_grad():
        y = model(y)[0 if embeddings else 1]

    # convert to numpy
    y = y.detach().cpu().numpy()

    return y

In [29]:
process_func(signal, sampling_rate)
#  Arousal    dominance valence
# [[0.5460759 0.6062269 0.4043165]]

#process_func(signal, sampling_rate, embeddings=True)
# Pooled hidden states of last transformer layer
# [[-0.00752167  0.0065819  -0.00746339 ...  0.00663631  0.00848747
#   0.00599209]]

torch.Size([1, 16000])


array([[0.54607475, 0.606226  , 0.40431628]], dtype=float32)

In [48]:
path='/content/drive/MyDrive/CSCI 535 Project/Implementation/training_models'
results=[]
for i,j in dataset.iterrows():
  current_file = path+j['file_path']
  sound = AudioSegment.from_wav('/content/drive/MyDrive/CSCI 535 Project/Implementation/training_models'+j['file_path'])
  sound = sound.set_channels(1)
  sound.export(current_file, format="wav")
  y, s = librosa.load(current_file, sr=16000) # Downsample 44.1kHz to 8kHz
  results.append(process_func(y.reshape(1,y.shape[0]), s))


torch.Size([1, 96000])
torch.Size([1, 96000])
torch.Size([1, 80000])
torch.Size([1, 176000])
torch.Size([1, 144000])
torch.Size([1, 112000])
torch.Size([1, 160000])
torch.Size([1, 64000])
torch.Size([1, 432000])
torch.Size([1, 879680])
torch.Size([1, 272000])
torch.Size([1, 80000])
torch.Size([1, 224000])
torch.Size([1, 224000])
torch.Size([1, 775680])
torch.Size([1, 439200])
torch.Size([1, 256000])
torch.Size([1, 336000])
torch.Size([1, 224000])
torch.Size([1, 930240])
torch.Size([1, 556480])
torch.Size([1, 520800])
torch.Size([1, 624000])
torch.Size([1, 832160])
torch.Size([1, 848480])
torch.Size([1, 913920])
torch.Size([1, 691040])
torch.Size([1, 400000])
torch.Size([1, 128000])
torch.Size([1, 942240])
torch.Size([1, 224000])
torch.Size([1, 852960])
torch.Size([1, 783200])
torch.Size([1, 112000])
torch.Size([1, 160000])
torch.Size([1, 886400])
torch.Size([1, 879040])
torch.Size([1, 861920])
torch.Size([1, 739360])
torch.Size([1, 804000])
torch.Size([1, 452480])
torch.Size([1, 300160

In [49]:
len(results) 
#memory error occurs for a video that is very long about 3 minutes, 29 seconds 

57

In [50]:
results.append(np.array([[1, 1 , 1 ]])) #appending a dummy vector for now, will have to shorten and extract clips from the video

In [51]:
results = results[:-1]

In [52]:
##  Arousal    dominance valence
arousal=[]
valence=[]
dominance=[]
for r in results:
  arousal.append(r[0][0])
  valence.append(r[0][1])
  dominance.append(r[0][2])

In [53]:
dataset['arousal']=arousal
dataset['valence']=valence
dataset['dominance']=dominance
dataset

,video_id,transcript,file_path,label,arousal,valence,dominance
0,audio_81.wav,I m not hating on men or blaming men I am lite...,/data/nonmisogyny_eval/audio_81.wav,0,0.630544,0.681015,0.262293
1,audio_82.wav,it seems like you guys fight it and I feel lik...,/data/nonmisogyny_eval/audio_82.wav,0,0.597634,0.625759,0.379943
2,audio_83.wav,okay so first of all I don't think its an atta...,/data/nonmisogyny_eval/audio_83.wav,0,0.672102,0.650475,0.466477
3,audio_84.wav,you also write about and this is unplesant stu...,/data/nonmisogyny_eval/audio_84.wav,0,0.359321,0.499993,0.416418
4,audio_85.wav,embattled CNN anchor Don Lemon absent for a th...,/data/nonmisogyny_eval/audio_85.wav,0,0.787427,0.782679,0.511144
5,audio_82.wav,she is wearing her fuck me boots very nice is ...,/data/misogyny_eval/audio_82.wav,1,0.394403,0.423904,0.391352
6,audio_83.wav,He hated girls who sat like boys who acted lik...,/data/misogyny_eval/audio_83.wav,1,0.311040,0.423604,0.078612
7,eval_12.wav,are you saying women cannot do condom hauls? W...,/data/misogyny_eval/eval_12.wav,1,0.772815,0.762862,0.217582
8,eval_5.wav,I don't think the world has ever been equal. I...,/data/misogyny_eval/eval_5.wav,1,0.773147,0.801821,0.276131
9,7NUU24OOh3Y,this video shows what the most popular books o...,/data/nonmisogyny_eval/7NUU24OOh3Y.wav,0,0.604059,0.633618,0.705374


In [54]:
dataset = dataset[['video_id', 'file_path', 'transcript', 'arousal', 'valence', 'dominance', 'label']]

In [55]:
dataset.to_csv("./data/eval_set.csv", index=False)

In [ ]:
df_eval=pd.read_csv('./data/eval_data.csv')
df_eval

,file_name,transcript,file_path,arousal,valence,dominance
0,eval_1.wav,can you please explain how to get over a break...,/data/misogyny_eval/eval_1.wav,0.690019,0.719820,0.346553
1,eval_2.wav,breakups are worse for men than they are for w...,/data/misogyny_eval/eval_2.wav,0.674508,0.741677,0.324086
2,eval_3.wav,she is going to be moving on and you are absol...,/data/misogyny_eval/eval_3.wav,0.738057,0.764024,0.242208
3,eval_4.wav,her revenge on you is instant. her revenge is ...,/data/misogyny_eval/eval_4.wav,0.778894,0.820213,0.127562
4,eval_5.wav,I don't think the world has ever been equal. I...,/data/misogyny_eval/eval_5.wav,0.773147,0.801821,0.276131
5,eval_6.wav,women's job was always procreation to look aft...,/data/misogyny_eval/eval_6.wav,0.751474,0.805252,0.297039
6,eval_7.wav,"In today's society with modern women, the most...",/data/misogyny_eval/eval_7.wav,0.570508,0.675798,0.431061
7,eval_8.wav,"now when I say loyal, I don't mean naughty but...",/data/misogyny_eval/eval_8.wav,0.678997,0.734515,0.307453
8,eval_9.wav,government was a real boys club back in the 70...,/data/misogyny_eval/eval_9.wav,0.571126,0.654537,0.577876
9,eval_10.wav,I think men are better than women. She's kiddi...,/data/misogyny_eval/eval_10.wav,0.647030,0.701312,0.309368


In [ ]:
df_eval['label']=len(df_eval)*[1]

In [ ]:
df_eval['label'].iloc[13:]=0

<ipython-input-34-c966b8bb1022>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eval['label'].iloc[13:]=0


In [ ]:
df_eval=df_eval.sample(frac=1)
df_eval

,file_name,transcript,file_path,arousal,valence,dominance,label
11,eval_12.wav,are you saying women cannot do condom hauls? W...,/data/misogyny_eval/eval_12.wav,0.772815,0.762862,0.217582,1
4,eval_5.wav,I don't think the world has ever been equal. I...,/data/misogyny_eval/eval_5.wav,0.773147,0.801821,0.276131,1
20,7NUU24OOh3Y,this video shows what the most popular books o...,/data/nonmisogyny_eval/7NUU24OOh3Y.wav,0.604059,0.633617,0.705374,0
5,eval_6.wav,women's job was always procreation to look aft...,/data/misogyny_eval/eval_6.wav,0.751474,0.805252,0.297039,1
6,eval_7.wav,"In today's society with modern women, the most...",/data/misogyny_eval/eval_7.wav,0.570508,0.675798,0.431061,1
8,eval_9.wav,government was a real boys club back in the 70...,/data/misogyny_eval/eval_9.wav,0.571126,0.654537,0.577876,1
3,eval_4.wav,her revenge on you is instant. her revenge is ...,/data/misogyny_eval/eval_4.wav,0.778894,0.820213,0.127562,1
16,IXpL7akTIPE,and that concludes our campus tour so how do y...,/data/nonmisogyny_eval/IXpL7akTIPE.wav,0.963565,0.903796,0.542248,0
24,IBi1pOAJaNo,because on the day I actually it was my first ...,/data/nonmisogyny_eval/IBi1pOAJaNo.wav,0.619481,0.640681,0.368204,0
12,eval_13.wav,women should clean up. and women should clean ...,/data/misogyny_eval/eval_13.wav,0.612304,0.738970,0.297879,1


In [58]:
df_eval.to_csv("./data/eval_data.csv", index=False)

NameError: ignored